In [1]:
import pandas as pd
import numpy as np
import nltk 

In [2]:
data=pd.read_csv("./chrome_reviews.csv")
data.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [3]:
df=data[['ID','Text','Star']]
df.head()

,ID,Text,Star
0,3886,This is very helpfull aap.,5
1,3887,Good,3
2,3888,Not able to update. Neither able to uninstall.,1
3,3889,Nice app,4
4,3890,Many unwanted ads,1


In [5]:
#Adding new column
df['Label'] = df['Star'].apply(lambda x: 1 if x >= 2 else 0)
df.head()

/var/folders/lv/hmttfrg13gx3kdn17n11y5600000gn/T/ipykernel_43212/4177472576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Star'].apply(lambda x: 1 if x >= 2 else 0)


,ID,Text,Star,Label
0,3886,This is very helpfull aap.,5,1
1,3887,Good,3,1
2,3888,Not able to update. Neither able to uninstall.,1,0
3,3889,Nice app,4,1
4,3890,Many unwanted ads,1,0


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Shikhar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# data cleaning and preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

p=PorterStemmer()

In [8]:
def words(data):
    l=[]
    for i in range(0,len(data)):
        q=re.sub("[^a-zA-Z]"," ",str(df['Text'][i]))
        q=q.lower()
        q=q.split()
        q= [p.stem(word) for word in q if not word in stopwords.words('english')]
        q=' '.join(q)
        l.append(q)
    for i in range(len(l)):
        data['Text'][i]=l[i]
    return data

In [9]:
df=words(df)
df.head()

/var/folders/lv/hmttfrg13gx3kdn17n11y5600000gn/T/ipykernel_43212/1211091213.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i]=l[i]


,ID,Text,Star,Label
0,3886,helpful aap,5,1
1,3887,good,3,1
2,3888,abl updat neither abl uninstal,1,0
3,3889,nice app,4,1
4,3890,mani unwant ad,1,0


In [11]:
def words2():
  global df
  # Removing empty string or ' ' in 'Text' after stemming if any.
  nan_value = float("NaN")
  df.replace("", nan_value, inplace=True)
  df=df.dropna()

  # Resetting the index
  df.reset_index(inplace=True)
  l=list(df['Text'])

In [12]:
words2()

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [14]:
X=df['Text'].values
y=df['Label'].values

# Splitting the dataset into train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [36]:
models,model_names,best_scores,best_params,test_score=[],[],[],[],[]

In [37]:
# Creating Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
text_clf_NB = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfNB', MultinomialNB())])

parameters_NB = {'vect__ngram_range': [(1, 1), (1, 2),(1,3)],'tfidf__use_idf': (True, False),'clfNB__alpha': [0.01 ,0.05,0.08, 0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}

Nb = GridSearchCV(text_clf_NB, parameters_NB, n_jobs=-1)
Nb = Nb.fit(X_train,y_train)

Nb_model=Nb.best_estimator_
y_pred=Nb_model.predict(X_test)

model_names.append('MultinomialNB')
models.append(Nb)
best_params.append(Nb.best_params_)
best_scores.append(Nb.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test Accuracy score : ",accuracy_score(y_test,y_pred))
print(Nb.best_score_)
print(Nb.best_params_)

Test Accuracy score :  0.8368080517613228
0.8417547363726271
{'clfNB__alpha': 0.1, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


In [38]:
# Creating model for Logistic regression
text_clf_Log = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfLog', LogisticRegression())])
parameters_Log = {'vect__ngram_range': [(1, 1), (1, 2),(1,3)],'tfidf__use_idf': (True, False),
                  'clfLog__penalty':['l1','l2','elasticnet','none'],'clfLog__C':[1.0,1.5,2.0],
                  'clfLog__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                  'clfLog__multi_class':['ovr']}
lr = GridSearchCV(text_clf_Log, parameters_Log, n_jobs=-1)
lr = lr.fit(X_train,y_train)

lr_model=lr.best_estimator_
y_pred=lr_model.predict(X_test)

model_names.append('LogisticRegression')
models.append(lr)
best_params.append(lr.best_params_)
best_scores.append(lr.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred)) 
print(lr.best_score_)
print(lr.best_params_)

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Shikhar/opt/anaconda3/lib/

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "

...

ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packag

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/

/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Shikhar/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("T

Test accuracy score :  0.8475916606757729
0.8394178996425502
{'clfLog__C': 2.0, 'clfLog__multi_class': 'ovr', 'clfLog__penalty': 'l2', 'clfLog__solver': 'liblinear', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}


In [39]:
for i in range(len(test_score)):
  print(model_names[i])
  print(best_scores[i])
  print(best_params[i])
  print('Test accuracy :',test_score[i])
  print('-'*50)

MultinomialNB
0.8417547363726271
{'clfNB__alpha': 0.1, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Test accuracy : 0.8368080517613228
--------------------------------------------------
LogisticRegression
0.8394178996425502
{'clfLog__C': 2.0, 'clfLog__multi_class': 'ovr', 'clfLog__penalty': 'l2', 'clfLog__solver': 'liblinear', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 3)}
Test accuracy : 0.8475916606757729
--------------------------------------------------


In [29]:
import joblib

In [41]:
for i in range(len(model_names)):
  if model_names[i]=='LogisticRegression':
    model=models[i]
joblib.dump(model,'model.mod')

['model.mod']